# Best Time to Buy and Sell Stock III

You are given an array `prices` where `prices[i]` is the price of a given stock on the <code>i<sup>th</sup></code> day.

Find the maximum profit you can achieve. You may complete at most two transactions.

Note: You may not engage in multiple transactions simultaneously (i.e., you must sell the stock before you buy again).

## Examples

**Example 1:**
```
Input: prices = [3,3,5,0,0,3,1,4]
Output: 6
```
Explanation: 
- Buy on day 4 (price = 0) and sell on day 6 (price = 3), profit = 3-0 = 3.
- Then buy on day 7 (price = 1) and sell on day 8 (price = 4), profit = 4-1 = 3.

**Example 2:**
```
Input: prices = [1,2,3,4,5]
Output: 4
```
Explanation: Buy on day 1 (price = 1) and sell on day 5 (price = 5), profit = 5-1 = 4.

Note that you cannot buy on day 1, buy on day 2 and sell them later, as you are engaging multiple transactions at the same time. You must sell before buying again.

**Example 3:**
```
Input: prices = [7,6,4,3,1]
Output: 0
```
Explanation: In this case, no transaction is done, i.e. max profit = 0.

## Method 1: Left-Right Partition

$\quad$ A classic approach is to split all transactions into two segments:
1. The first transaction only occurs in the “left” part of the range (from day `0` to day `i`).
2. The second transaction only occurs in the remaining “right” part (from day `i+1` to the end).

Then, for every possible partition point `i`:
1. Calculate the maximum profit from at most one transaction in the left segment (days `0` to `i`), denoted as `leftProfit[i]`.
2. Calculate the maximum profit from at most one transaction in the right segment (days `i+1` to the end), denoted as `rightProfit[i+1]`.

$\quad$ When you iterate over all possible `i`, `leftProfit[i] + rightProfit[i+1]` represents the total profit if the first transaction ends before or on day `i` and the second one starts afterward. The maximum sum across all `i` is the maximum total profit achievable with at most two transactions.

$\quad$ As for how to calculate `leftProfit` and `rightProfit`, we can use the same approach as in the previous problem "Best Time to Buy and Sell Stock".

In [3]:
'''
    Time Complexity: O(n)
    Space Complexity: O(n)
'''

class Solution:
    def maxProfit(self, prices: list[int]) -> int:
        n = len(prices)
        if n < 2:
            return 0

        left_profits = [0] * n
        buy_price = prices[0]
        max_profit = 0
        for i, price in enumerate(prices):
            if buy_price <= price:
                max_profit = max(max_profit, price - buy_price)
            else:
                buy_price = price
            left_profits[i] = max_profit

        right_profits = [0] * n
        sell_price = prices[-1]
        max_profit = 0
        for i in range(n - 1, -1, -1):
            if prices[i] < sell_price:
                max_profit = max(max_profit, sell_price - prices[i])
            else:
                sell_price = prices[i]
            right_profits[i] = max_profit

        return max(
            max(left_profits[i] + right_profits[i + 1] for i in range(n - 1)),
            left_profits[-1],
        )

$\quad$ The above algorithm can also be reinterpreted from the perspective of dynamic programming.

1. $leftProfit$ Array:

- Traverse from left to right.
- $leftProfit[i]$ = maximum profit you can get by one transaction ending on or before day $i$.
- We can maintain a running minimum price so far (minPriceSoFar) and update:
    $$leftProfit[i]=\max(leftProfit[i−1],prices[i]−minPriceSoFar)$$
    and also update minPriceSoFar if we find a new lower price on day $i$. 

Note that $leftProfit[i−1]$ corresponds to the scenario where no selling action is performed on day $i$ and $prices[i]−minPriceSoFar$ corresponds to the scenario where a selling action is performed on day $i$.

2. $rightProfit$ Array:
- Traverse from right to left.
- $rightProfit[i]$ = maximum profit you can get by one transaction starting on or after day $i$.
- Maintain a running maximum price so far (maxPriceSoFar), and update:
    $$rightProfit[i]=\max(rightProfit[i+1],maxPriceSoFar−prices[i]),$$
    and update maxPriceSoFar if we find a new higher price on day $i$.

## Method 2

$\quad$ If we count buying and selling on the same day (which yields zero profit) as one transaction, then having at most two transactions essentially means having exactly two transactions. At the same time, we allow unlimited buy and sell transactions in a single day (of course, you must sell before buying again). This will not affect the result because the profit from buying and selling on the same day is zero.

$\quad$ For any $i$, let $buy_1[i]$ represent the maximum net profit that can be obtained after making the first stock purchase within the first $i+1$ days, let $sell_1[i]$ represent the maximum net profit that can be obtained after making the first stock sale within the first $i+1$ days, let $buy_2[i]$ represent the maximum net profit that can be obtained after making the second stock purchase within the first $i+1$ days, and let $sell_2[i]$ represent the maximum net profit that can be obtained after making the second stock sale within the first $i+1$ days.

$\quad$ It is clear that $buy_1[0] = - prices[0] = \max(-\infty, - prices[0])$. For any $i > 0$, we have two options for $buy_1[i]$:
- The maximum net profit that can be obtained after completing the first stock purchase within the first $i+1$ days, but not making the purchase on the $(i+1)$-th day (i.e., the say with index $i$), is $buy_1[i-1]$;
- The maximum net profit that can be obtained after completing the first stock purchase within the first $i+1$ days, and making the purchase on the $(i+1)$-th day, is $-prices[i]$.

Hence 
$$buy_1[i] = \max(buy_1[i-1], -prices[i]).$$

$\quad$ Next we consider $sell_1$. It is clear that $sell_1[0] = 0 = \max(0, buy_1[0] + prices[0])$. For any $i > 0$, we have two options for $sell_1[i]$:
- The maximum net profit that can be obtained after completing the first stock sale within the first $i+1$ days, but not making the sale on the $(i+1)$-th day (i.e., the say with index $i$), is $sell_1[i-1]$;
- The maximum net profit that can be obtained after completing the first stock sale within the first $i+1$ days, and making the sale on the $(i+1)$-th day, is $buy_1[i] + prices[i]$ (note that we have covered the scenario where a stock is bought and immediately sold on the $i$-th day).

Hence we have 
$$sell_1[i] = \max(sell_1[i-1], buy_1[i] + prices[i]).$$

$\quad$ Then we consider $buy_2$. It is clear that $buy_2[0] = - prices[0] = \max(-\infty, sell_1[0] - prices[0])$ (buy on the first day, sell on the same day, and then buy again). For any $i > 0$, we have two options for $buy_2[i]$:
- The maximum net profit that can be obtained after completing the second stock purchase within the first $i+1$ days, but not making the purchase on the $(i+1)$-th day (i.e., the say with index $i$), is $buy_2[i-1]$;
- The maximum net profit that can be obtained after completing the second stock purchase within the first $i+1$ days, and making the purchase on the $(i+1)$-th day, is $sell_1[i] - prices[i]$.

Hence
$$buy_2[i] = \max(buy_2[i-1], sell_1[i] - prices[i]).$$

$\quad$ Finally, we consider $sell_2$. It is clear that $sell_2[0] = 0 = \max(0, buy_2[0] + prices[0])$. For any $i > 0$, we have two options for $sell_2[i]$:
- The maximum net profit that can be obtained after completing the second stock sale within the first $i+1$ days, but not making the sale on the $(i+1)$-th day (i.e., the say with index $i$), is $sell_2[i-1]$;
- The maximum net profit that can be obtained after completing the second stock sale within the first $i+1$ days, and making the sale on the $(i+1)$-th day, is $buy_2[i] + prices[i]$.

Hence we have
$$sell_2[i] = \max(sell_2[i-1], buy_2[i] + prices[i]).$$

$\quad$ In summary, if we set $buy_1[-1] = buy_2[-1] = -\infty$ and $sell_1[-1] = sell_2[-1] = 0$, then we have the following recurrence relations:
$$
\begin{cases}
buy_1[i] = \max(buy_1[i-1], -prices[i]), \\
sell_1[i] = \max(sell_1[i-1], buy_1[i] + prices[i]), \\
buy_2[i] = \max(buy_2[i-1], sell_1[i] - prices[i]), \\
sell_2[i] = \max(sell_2[i-1], buy_2[i] + prices[i]).
\end{cases}
$$
What we want is $sell_2[n-1]$.


In [4]:
class Solution:
    def maxProfit(self, prices: list[int]) -> int:
        if not prices:
            return 0
        n = len(prices)
        buy_1, buy_2 = [float("-inf")] * (n + 1), [float("-inf")] * (n + 1)
        sell_1, sell_2 = [0] * (n + 1), [0] * (n + 1)

        for i in range(1, n + 1):
            buy_1[i] = max(buy_1[i - 1], -prices[i - 1])
            sell_1[i] = max(sell_1[i - 1], buy_1[i] + prices[i - 1])
            buy_2[i] = max(buy_2[i - 1], sell_1[i] - prices[i - 1])
            sell_2[i] = max(sell_2[i - 1], buy_2[i] + prices[i - 1])

        return sell_2[n]

        return sell2

$\quad$ Noticing that $buy_1[i]$ only depends on $buy_1[i-1]$, $sell_1[i]$ only depends on $sell_1[i-1]$ and $buy_1[i]$, $buy_2[i]$ only depends on $buy_2[i-1]$ and $sell_1[i]$, and $sell_2[i]$ only depends on $sell_2[i-1]$ and $buy_2[i]$, we can reduce the space complexity to $O(1)$.

In [ ]:
class Solution:
    def maxProfit(self, prices: list[int]) -> int:
        if not prices:
            return 0

        buy1 = float('-inf')
        sell1 = 0
        buy2 = float('-inf')
        sell2 = 0

        for price in prices:
            buy1 = max(buy1, -price)
            sell1 = max(sell1, buy1 + price)
            buy2 = max(buy2, sell1 - price)
            sell2 = max(sell2, buy2 + price)

        return sell2
